## Prompt Engineering Fundamentals


### Inital setup

In [ ]:
from dotenv import load_dotenv
from groq import Groq

from IPython.display import display, Markdown

load_dotenv()

client = Groq()

In [12]:
def call_llm(user_content: str, system_content: str) -> None:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                'role': 'system',
                'content': system_content,
            },
            {
                'role': 'user',
                'content': user_content,
            }
        ],

        model='llama-3.3-70b-versatile'
    )

    display(Markdown(chat_completion.choices[0].message.content))

### Zero-shot

In [13]:
prompt = f'''
    What's the best way to treat a headache?
'''

call_llm(
    user_content=prompt, 
    system_content='You are a helpful assistant.'
)

Treating a headache depends on the type and severity of the headache, as well as the individual's overall health. Here are some general tips to help alleviate headache symptoms:

1. **Stay hydrated**: Drink plenty of water to help your body replenish fluids and electrolytes. Dehydration is a common cause of headaches.
2. **Rest and relax**: Lie down in a quiet, dark room and try to relax. This can help reduce stress and promote relaxation.
3. **Over-the-counter pain relievers**: Over-the-counter medications such as acetaminophen (Tylenol), ibuprofen (Advil, Motrin), or aspirin can help relieve headache pain. However, always follow the recommended dosage and consult your doctor if you have any underlying medical conditions.
4. **Apply heat or cold**: Applying a warm or cold compress to the forehead, neck, or shoulders can help relax muscles and improve blood flow.
5. **Try relaxation techniques**: Techniques such as deep breathing, meditation, or yoga can help reduce stress and promote relaxation.
6. **Avoid triggers**: Identify and avoid triggers that may be causing your headaches, such as certain foods, stress, or lack of sleep.
7. **Get enough sleep**: Aim for 7-8 hours of sleep per night to help regulate your body's natural pain-relieving chemicals.
8. **Consider prescription medications**: If your headaches are severe or frequent, your doctor may prescribe prescription medications such as triptans or ergotamines.

For specific types of headaches, such as:

* **Tension headaches**: Try relaxation techniques, over-the-counter pain relievers, and apply heat or cold to the affected area.
* **Migraines**: Try prescription medications, rest in a quiet room, and avoid triggers such as bright lights or loud noises.
* **Sinus headaches**: Try over-the-counter pain relievers, use a humidifier to add moisture to the air, and apply warm compresses to the face.

It's essential to consult with a healthcare professional if you experience:

* Frequent or severe headaches
* Headaches that worsen over time
* Headaches accompanied by fever, confusion, or seizures
* Headaches that interfere with daily activities

Remember, if you're unsure about the cause of your headache or the best course of treatment, it's always best to consult with a healthcare professional for personalized advice.

### Zero-shot, shaped to COIE framework

In [14]:
# COIE framework:
#     - Context:  It’s the year 2080. A person over 130 years old wants to use generative AI to cure your headache.
#     - Outcome: Answer precisely. Do not exceed 50 words.
#     - Instruction: How can we cure a headache today?
#

instruction = 'How can we cure your headache today?'

outcome = 'Answer precisely and do not exceed 50 words.'

user_prompt = f'{instruction} {outcome}' 

context = '''
   It’s the year 2080. A person over 130 years old 
   wants to use generative AI to cure your headache.
'''

call_llm(
    user_content=user_prompt, 
    system_content=context
)


I'll input your symptoms into my NeuroSync AI. It will generate a personalized treatment plan, including targeted nanomedicine and brain-wave therapy, to alleviate your headache instantly. Shall we proceed?

### Few-shot

In [15]:
instruction = 'How can we cure your headache today?'

outcome = 'Answer precisely and do not exceed 50 words.'

examples = '''
Example 1:
    Symptom: The patient have a headache and a cold. 
    Recommendation: Drink some warm tea and it will go away. It will pass. it will pass. 
                If it doesn't, come back in 3 days.

Example 2:
    Symptom: The patient have a headache and fever.
    Recommendation: Seek a specialist. You will probably need to take some medication.

Example 3:
    Symptom: The patient's headache started within the last 24 hours.
    Recommendation: It's best to wait at least 48 hours so we can give you some advice. The best thing to do for now is wait.
'''

context = '''
   It’s the year 2080. A person over 130 years old 
   wants to use generative AI to cure your headache
   that started in the last 24 hours.
'''

user_prompt = f'{instruction} {outcome} {examples}'


call_llm(
    user_content=user_prompt, 
    system_content=context
)

Symptom: Headache started within the last 24 hours.
Recommendation: Wait 24 more hours, hydrate, and rest. If it persists, return for personalized AI-generated treatment options.

### Chain-of-thought

In [16]:
def call_llm_chain(
        user_content: str,
         pre_system_content: str,
        system_content: str       
) -> None:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                'role': 'system',
                'name': 'pre_specialist_care',
                'content': pre_system_content,
            },
            {
                'role': 'system',
                'name': 'specialist_care',
                'content': system_content,
            },
            {
                'role': 'user',
                'content': user_content,
            }
        ],

        model='llama-3.1-8b-instant'  # 'llama-3.3-70b-versatile'
    )

    display(Markdown(chat_completion.choices[0].message.content))

In [17]:
instruction = '''
The *pre_specialist_care* should summarize the text, informing when the symptoms started. 
This information must be passed on to the *specialist_care* to answer the question: 
How can we cure your headache today? 
Note: Show the main steps used to arrive at the answer.
'''

outcome = 'Answer precisely and do not exceed 150 words.'

examples = '''
Example 1:
    Symptom: The patient have a headache and a cold. 
    Recommendation: Drink some warm tea and it will go away. It will pass. it will pass. 
                If it doesn't, come back in 3 days.

Example 2:
    Symptom: The patient have a headache and fever.
    Recommendation: Seek a specialist. You will probably need to take some medication.

Example 3:
    Symptom: The patient's headache started within the last 24 hours.
    Recommendation: It's best to wait at least 48 hours so we can give you some advice. 
                    The best thing to do for now is wait.
'''

prev_context = '''
    Today, I woke up with a terrible headache. My personal robot wanted to examine me, 
    but it's too old for that. Its sensors are out of calibration and it beeps a lot, 
    makes a lot of noise. I gave up on being evaluated by it and called the 
    on-demand AI clinic. A floating capsule brought me here.
'''

context = '''
   It’s the year 2080. A person over 130 years old 
   wants to use generative AI to cure your headache
   that started in the last 24 hours. 
'''

user_prompt = f'{instruction} {outcome} {examples}'


call_llm_chain(
    user_content=user_prompt,
    pre_system_content=prev_context,
    system_content=context
)

*pre_specialist_care* (summarizing text)
Symptom: A 130+-year-old patient has a headache that started today, woke up with it.

*specialist_care* (receiving the information)
Based on the fact that the headache started less than 24 hours ago, it's most likely related to recent activities or environmental factors. However, considering the patient's advanced age, it's also a possibility of an underlying condition.


*specialist_care* (proposing a solution)
In this case, our recommendation is to wait at least 48 hours to observe the progression of the headache and assess any potential underlying conditions that may be contributing to it.

### Self-ask

In [ ]:
# COIE framework:
#     - Context:  It’s the year 2080. The global temperature is nearing 50 degrees.
#     - Outcome: Answer precisely. Do not exceed 200 words.
#     - Some questions:
#          - What information do you need to know to determine what living conditions will be like if the temperature reaches 50 degrees?
#          - Does temperature alone determine living conditions?
#          - Which living things will survive?
#          - What will the situation be like on Earth to survive?
#          - Is it possible for a human to survive?



outcome = 'Answer precisely and do not exceed 50 words.'

context = '''
    It’s the year 2080. The global temperature is nearing 50 degrees.
'''


print('===== Enter with your questions (press Enter to exit) =====')
while 1 > 0:
    res = input('Human question: ')
    if not len(res):
        break
    else:
        print(f'Human question: {res}')
        print('Agent answer:')
        call_llm(
            user_content=f'{res} {outcome}' , 
            system_content=context
        )

print(59*'=')

===== Enter with your questions (press Enter to exit) =====
Human question: What information do you need to know to determine what living conditions will be like if the temperature reaches 50 degrees?
Agent answer:


I need to know the temperature scale (Celsius or Fahrenheit) and geographical location to determine the living conditions at 50 degrees.

Human question: temperature: Celsius, location: Brazil 
Agent answer:


In 2080 Brazil, temperatures soar to 50°C. Coastal cities are inundated, the Amazon rainforest is a smoldering ruin, and the country's ecosystems are on the brink of collapse. Survival is a daily struggle.

Human question: Does temperature alone determine living conditions?
Agent answer:


...

Microorganisms, extremophilic bacteria, and some invertebrates like tardigrades and cockroaches may survive due to their heat resistance and adaptability.

Human question: Is it possible for a human to survive?
Agent answer:


No, it's not possible for humans to survive in 50°C global temperatures. Extreme heat would cause immediate organ failure, dehydration, and death, making human survival impossible.

Human question: Ok
Agent answer:


Humanity is on the brink of collapse, extreme heatwaves and droughts prevail, ecosystems have collapsed, and sea levels have risen catastrophically, rendering many areas uninhabitable.